In [1]:
import pandas as pd
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import lightgbm as lgb
import altair as alt
from altair.expr import datum

In [2]:
train = pd.read_csv(r'D:\HPDemo\Projects\Kaggle_house_price\train.csv')
test = pd.read_csv(r'D:\HPDemo\Projects\Kaggle_house_price\test.csv')

In [3]:
print("The train data size before dropping Id feature is : {} ".format(train.shape))
print("The test data size before dropping Id feature is : {} ".format(test.shape))

#Save the 'Id' column
train_ID = train['Id']
test_ID = test['Id']

#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

#check again the data size after dropping the 'Id' variable
print("\nThe train data size after dropping Id feature is : {} ".format(train.shape)) 
print("The test data size after dropping Id feature is : {} ".format(test.shape))

The train data size before dropping Id feature is : (1460, 81) 
The test data size before dropping Id feature is : (1459, 80) 

The train data size after dropping Id feature is : (1460, 80) 
The test data size after dropping Id feature is : (1459, 79) 


In [4]:
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

In [5]:
train["SalePrice"] = np.log1p(train["SalePrice"])

ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (2917, 79)


D:\Anaconda\envs\classic_jl_lgbm\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [6]:
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

,Missing Ratio
PoolQC,99.691464
MiscFeature,96.400411
Alley,93.212204
Fence,80.425094
FireplaceQu,48.680151
LotFrontage,16.660953
GarageQual,5.450806
GarageCond,5.450806
GarageFinish,5.450806
GarageYrBlt,5.450806


In [7]:
all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data = all_data.drop(['Utilities'], axis=1)
all_data["Functional"] = all_data["Functional"].fillna("Typ")
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})

In [8]:
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)


#Changing OverallCond into a categorical variable
all_data['OverallCond'] = all_data['OverallCond'].astype(str)


#Year and month sold are transformed into categorical features.
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')

# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape))

all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: x.dropna().skew()).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

Shape all_data: (2917, 78)

Skew in numerical features: 



,Skew
MiscVal,21.950962
PoolArea,17.697766
LotArea,13.116240
LowQualFinSF,12.090757
3SsnPorch,11.377932
LandSlope,4.975813
KitchenAbvGr,4.302763
BsmtFinSF2,4.146636
EnclosedPorch,4.004404
ScreenPorch,3.947131


In [9]:
skewness = skewness[abs(skewness.Skew) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)

There are 36 skewed numerical features to Box Cox transform


In [10]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(2917, 220)


In [11]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [12]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

D:\Anaconda\envs\classic_jl_lgbm\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [13]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
    
    def fit(self, X, y):
        self.models_ = [x for x in self.models]
        
        for model in self.models_:
            model.fit(X, y)
        
        return self
    
    def predict(self, X):
        prediction = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        
        return np.mean(prediction, axis=1)

In [14]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
    
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=42)
        oof = np.zeros((X.shape[0], len(self.base_models)))
        
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                oof[holdout_index, i] = y_pred
            
        self.meta_model_.fit(oof, y)
        return self
    
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([
                model.predict(X) for model in base_models]).mean(axis=1)
                for base_models in self.base_models_
        ])
        return self.meta_model_.predict(meta_features)

In [15]:
from sklearn.linear_model import Ridge, Lasso, MultiTaskLasso, ElasticNet, BayesianRidge, LassoLars, SGDRegressor, HuberRegressor
from sklearn.svm import SVR, NuSVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.externals import joblib

In [32]:
clf_ridge = Ridge(random_state=42)
clf_lasso = Lasso(random_state=42)
clf_Enet = ElasticNet(random_state=42)
clf_Bayes = BayesianRidge()
clf_lassolars = LassoLars()

classifiers_one_names = ['Ridge', 'Lasso', 'ElasticNet', 'BayesianRidge', 'LassoLars']

classifiers_one = [clf_ridge, clf_lasso, clf_Enet, clf_Bayes, clf_lassolars]

In [ ]:
#download presaved models

for ix, clf in enumerate(classifiers_one):
    classifiers_one[ix] = joblib.load(str(classifiers_one_names[ix]) + '.joblib')

In [33]:
params_one = {
    0: {'clf__alpha': np.linspace(0.9, 1, 101)},
    1: {'clf__alpha': np.linspace(0.0001, 0.002, 101)},
    2: {'clf__alpha': np.linspace(0.0001, 0.004, 101), 'clf__l1_ratio': np.linspace(0.7, 0.8, 21)},
    3: {'clf__alpha_1': np.linspace(1e-9, 1e-7, 20), 'clf__alpha_2': np.linspace(0.0001, 0.1, 21)},
    4: {'clf__alpha': np.linspace(0.00001, 0.001, 101)}
}

In [34]:
%%time

best_params_one = {}

for ix, model in enumerate(classifiers_one):
    
    pipeline = Pipeline([
        ('robust', RobustScaler()),
        ('clf', model)
    ])
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    CV = GridSearchCV(pipeline, param_grid=params_one[ix], cv=kf, verbose=True, n_jobs=-1)
    CV.fit(train.values, y_train)
    classifiers_one[ix] = CV.best_estimator_.steps[1][1]

Fitting 5 folds for each of 101 candidates, totalling 505 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done 505 out of 505 | elapsed:   18.9s finished


Fitting 5 folds for each of 101 candidates, totalling 505 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   33.2s
[Parallel(n_jobs=-1)]: Done 505 out of 505 | elapsed:   36.4s finished


Fitting 5 folds for each of 2121 candidates, totalling 10605 fits


...

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   29.3s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   49.4s
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 2100 out of 2100 | elapsed:  2.2min finished


Fitting 5 folds for each of 101 candidates, totalling 505 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   17.0s


Wall time: 14min 31s


[Parallel(n_jobs=-1)]: Done 505 out of 505 | elapsed:   18.9s finished


In [35]:
# save models if necessary

for i in classifiers_one:
    joblib.dump(i, (str(i).split('(')[0] + '.joblib'))

In [17]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [81]:
score_one =[]
for i in classifiers_one:
    score_one.append(rmsle_cv(i).mean())

In [3]:
average = AveragingModels(models=classifiers_one)
print(rmsle_cv(average).mean())

NameError: name 'AveragingModels' is not defined

In [18]:
clf_huber = HuberRegressor()
clf_SVR = SVR()
clf_tree = RandomForestRegressor(random_state=42)

classifiers_two_names = ['HuberRegressor', 'SVR', 'RandomForestRegressor']

classifiers_two = [clf_huber, clf_SVR, clf_tree]

In [30]:
#download presaved models if necessary

for ix, clf in enumerate(classifiers_two):
    classifiers_two[ix] = joblib.load(str(classifiers_two_names[ix]) + '.joblib')

In [21]:
params_two = {
    0: {'clf__alpha': np.linspace(0.0001, 1, 11), 'clf__epsilon': [x for x in range(1, 11)]},
    1: {'clf__kernel': ['linear'], 'clf__C': np.linspace(0.01, 1, 11), 'clf__epsilon': [x for x in range(11)]},
    2: {'clf__max_depth': [5, 6, 8, 10, 12, 20], 'clf__min_samples_split': [2, 3, 5, 10, 20, 30], 'clf__min_samples_leaf': [2, 3, 5, 10, 20, 30],
        'clf__n_estimators': [20]}
}

best_params_two = {}


for ix, model in enumerate(classifiers_two):
    
    pipeline = Pipeline([
        ('robust', RobustScaler()),
        ('clf', model)
    ])
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    CV = GridSearchCV(pipeline, param_grid=params_two[ix], cv=kf, verbose=True, n_jobs=-1)
    CV.fit(train.values, y_train)
    classifiers_two[ix] = CV.best_estimator_.steps[1][1]

Fitting 5 folds for each of 110 candidates, totalling 550 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   54.2s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 550 out of 550 | elapsed:  2.3min finished


Fitting 5 folds for each of 121 candidates, totalling 605 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   51.6s
[Parallel(n_jobs=-1)]: Done 568 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 605 out of 605 | elapsed:  3.6min finished


Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   43.5s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:  1.9min finished


In [35]:
# save models if necessary

for i in classifiers_two:
    joblib.dump(i, (str(i).split('(')[0] + '.joblib'))

In [ ]:
%%time
score_two =[]
for i in classifiers_two:
    score_two.append(rmsle_cv(i).mean())

In [ ]:
average = AveragingModels(models=classifiers_two)
print(rmsle_cv(average).mean(), score_two)